In [108]:
import os
import json
import pandas as pd
from datetime import datetime, timedelta
from collections import defaultdict

directory = os.fsencode('data/wtt_matches')

In [109]:
tf = pd.read_csv('data/tournaments_wtt.tsv', sep='\t', parse_dates=['StartDateTime', 'EndDateTime'])

In [110]:
players = []
genders = defaultdict(lambda: None)

for row in tf.itertuples():
    evt = os.path.join(f'data/wtt_matches', str(row.EventId))
    if not os.path.isdir(evt):
        continue

    for filename in os.listdir(evt):
        if not filename.endswith(".json"):
            continue

        m = None
        with open(os.path.join(f'data/wtt_matches', str(row.EventId), filename)) as f:
            m = json.load(f)

        gender = m['documentCode'][3]

        isTeam = False
        for c in m['competitiors']:
            if isTeam:
                break

            for p in c['players']:
                p['gender'] = gender
                pid = int(p['playerId'])
                if pid > 100000000 or pid < 100000:
                    isTeam = True
                if isTeam:
                    break

                if not p['playerOrgCode']:
                    p['playerOrgCode'] = c['competitiorOrg']
                players.append(p)

        if isTeam and m['teamParentData']:
            for _m in m['teamParentData']['extended_info']['matches']:
                if not _m.get('match_result'):
                    continue

                mm = _m['match_result']
                for c in mm['competitiors']:
                    for p in c['players']:
                        p['gender'] = gender
                        pid = int(p['playerId'])
                        if pid > 100000000 or pid < 100000:
                            continue

                        if not p['playerOrgCode']:
                            p['playerOrgCode'] = c['competitiorOrg']
                        players.append(p)


In [111]:
pf = pd.DataFrame(players)
pf.drop(columns=['playerPosition'], inplace=True)
pf

,playerId,playerName,playerGivenName,playerFamilyName,playerOrgCode,gender
0,111115,AGUIRRE Marcelo,None,None,PAR,M
1,114247,WANG Eugene,None,None,CAN,M
2,109777,VEGA Paulina,None,None,CHI,W
3,132083,DIAZ Fabiola,None,None,PUR,W
4,117294,ORTEGA Daniela,None,None,CHI,W
...,...,...,...,...,...,...
44821,137627,LIU Yangzi,None,None,AUS,W
44822,123980,HARIMOTO Tomokazu,None,None,JPN,M
44823,100696,ASSAR Omar,None,None,EGY,M
44824,114715,QIU Dang,None,None,GER,M


In [112]:
pf[pf.playerName == 'Lubomir PISTEJ']

,playerId,playerName,playerGivenName,playerFamilyName,playerOrgCode,gender
1543,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,X
1732,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,M
1773,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,M
1817,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,M
2157,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,M
2491,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,M


In [113]:
pf[pf.playerId.isna()]
# only 2, manually fix

,playerId,playerName,playerGivenName,playerFamilyName,playerOrgCode,gender


In [114]:
name_conflicts = pf.groupby('playerId')['playerName'].apply(set)
conf = pd.DataFrame(name_conflicts)
conf['cnt'] = name_conflicts.apply(len)
conf[conf.cnt > 1]


,playerName,cnt
playerId,,
100032,"{ABDEL-AZIZ Farah, Farah ABDELAZIZ}",2
100439,"{Ahmed SALEH, SALEH Ahmed}",2
100486,"{ALTO Gaston, Gaston ALTO}",2
100621,"{Tiago Apolonia, APOLONIA Tiago, Tiago APOLONIA}",3
100696,"{Omar ASSAR, Omar Assar, ASSAR Omar}",3
...,...,...
200310,"{ADEGOKE Muizz, ADEGOKE Muizz Olawale}",2
200361,"{RIOS Carlos, RIOS ROMERO Carlos Josue Alberto}",2
200391,"{MENEZES ARAUJO Abimael, MENEZES Abimael}",2


In [115]:
id_conflicts = pf.groupby('playerName')['playerId'].apply(set)
idconf = pd.DataFrame(id_conflicts)
idconf['cnt'] = id_conflicts.apply(len)
idconf[idconf.cnt > 1]
# These actually might be different people... 

,playerId,cnt
playerName,,
LEE Daeun,"{132702, 135391}",2
Yang WANG,"{109995, 112735}",2


In [116]:
org_conflicts = pf.groupby('playerId')['playerOrgCode'].apply(set)
oconf = pd.DataFrame(org_conflicts)
oconf['cnt'] = org_conflicts.apply(len)
oconf[oconf.cnt > 1]


,playerOrgCode,cnt
playerId,,
100868,"{LUX/SVK, CZE/SVK, SVK/CZE, SVK/FRA, SVK/USA, ...",7
102380,"{ENG/GER, GBR, ENG}",3
103163,"{USA, USA/SUI}",2
103903,"{ROU, ROU/ESP}",2
104359,"{SRB, SVK/SRB}",2
...,...,...
137627,"{AUS, SVK/AUS, FRA/AUS, NOR/AUS, ESP/AUS, AUS/...",7
146048,"{PER, PER/ECU}",2
200322,"{LUX/KAZ, KAZ}",2


In [117]:
cleaned = []
for id, rows in pf.groupby('playerId'):
    clrow = {
        'id': id,
        'org': None,
        'name': '',
        'gender': 'X',
    }
    for row in rows.itertuples():
        if row.gender != 'X':
            clrow['gender'] = row.gender
        if clrow['name'] != row.playerName and min(sum(1 for c in clrow['name'] if c.isupper()), 4) < min(sum(1 for c in row.playerName if c.isupper()), 4):
            clrow['name'] = row.playerName
        if not clrow['org']:
            clrow['org'] = row.playerOrgCode
        elif row.playerOrgCode:
            # take shorter country code, don't want doubles codes
            if len(row.playerOrgCode) < len(clrow['org']):
                clrow['org'] = row.playerOrgCode

    cleaned.append(clrow)
cf = pd.DataFrame(cleaned)

In [118]:
pf[pf.playerId.isin(cf[cf.org.isna()].id)]

,playerId,playerName,playerGivenName,playerFamilyName,playerOrgCode,gender


In [119]:
cf.to_csv('data/wtt_cleaned/players.tsv', index=False, sep='\t')